# Biostatistics Tutorial 

Inspired by: 
- Pollard et al (2019), "Empowering Statistical Methods for Cellular and Molecular Biologists",  Molecular Biology of the CellVol. 30, No. 12 https://www.molbiolcell.org/doi/full/10.1091/mbc.E15-02-0076

# Introduction
In this tutorial, python is used to analyze six data sets (provided as Excel files), each using a different statistical test:

1. Fisher Exact test for association between binary variables
2. Welch's t-test for unequal variances
3. Student's t-test on log transformed responses
4. Logistic regression for categorical response and two treatment variables
5. Chi-Square contingency test on combined response and combined treatment groups
6. ANOVA with Tukey-Kramer post-hoc

For each example, the following steps are taken:
-  State biological hypothesis
-  State the number and types of variables
-  Determine the preferred statistical test and null hypothesis
-  Check if data meet the assumptions of the preferred statistical test
-  Decide what statistical test to use
-  Run the statistical test
-  Interpret the results of the statistical test
-  Display the data and statistical results in a figure

You do not need to install python or perform any of the analyses in the tutorial in order to learn from the examples.

### Installing python locally

Installing Python is generally easy, and nowadays many Linux and UNIX distributions include a recent Python. Even some Windows computers (notably those from HP) now come with Python already installed. If you do need to install Python and aren't confident about the task you can find a few notes on the BeginnersGuide/Download wiki page, but installation is unremarkable on most platforms.


### First steps in python

You can find a simple tutorial focusing on the essentials you need to know to start programming with Python.
https://realpython.com/python-first-steps/

### Downloading and using Jupyter
Jupyter is free software for building data analysis notebooks with embedded programming language code, such as python or R. This tutorial was created in Jupyter and can be run as a live notebook with executable and editable python code if Jupyter is installed.

To install Jupyter, following the instructions on this page: https://jupyter.org/install
After Jupyter is installed, you will also need to allow Jupyter to run R by following the instructions on this page: https://anaconda.org/chdoig/jupyter-and-conda-for-r/notebook

### Tips on reading and using the tutorial
The tutorial is formatted with narrative text like this, python code blocks in grey boxes (see just below), and output from running the python code (including plots) just after the python code blocks.

In [ ]:
# this line that starts with the pound sign is a comment in a block of python code and does not get run
print('this is the output from some python code')

If you are running Jupyter, the python code throughout the tutorial can be executed using the "Run" button (or pressing shift + enter) and can also be edited.

### Loading packages
Although you only have to install a specialized package once, you have to load the package every time you restart python and want to use functions in the package.  Packages or modules can be loaded using the import statement as follows:

In [ ]:
import pandas as pd
from scipy import stats as st
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt

# we will use bioinfokit (v1.0.3 or later) for performing tukey HSD test
# check documentation here https://github.com/reneshbedre/bioinfokit
from bioinfokit.analys import stat

# Example 0: Fisher Exact test for binary variables

In this first example, we are testing the biological hypothesis that a mutant genotype affects a phenotype we are measuring. Our statistical null hypothesis is that genotype has no effect on the phenotype.

There are two variables in the experiment: Genotype and Phenotype. Genotype is a categorical variable with two possible values: WT and Mutant. Phenotype is dicothomous variable (Pizza or Pasta). Phenotype is our response variable. Genotype is our treatment variable.

Based on these two variables, we will run a Fisher Exact Test to check if they are significantly associated or not.

In [ ]:
import random
geno = ['WT']*100 + ['Mutant']*100
pheno = ['Pizza']*50 + ['Pasta']*50 + ['Pizza']*72 + ['Pasta']*28
df = pd.DataFrame({'Genotype':geno,'Phenotype':pheno})
df = df.iloc[np.random.permutation(200)]
df.to_excel('dataset0.xlsx',index=False)

In [ ]:
# read fist data file into data frame object
data0 = pd.read_excel('dataset0.xlsx')
# note that the above line of code assumes the data file is in the same folder as the notebook
data0.head()

In [ ]:
table = pd.crosstab(data0.Phenotype,data0.Genotype)
table

In [ ]:
sns.histplot(data=data0,x='Genotype',hue='Phenotype',multiple='dodge',shrink=0.5)

In [ ]:
st.fisher_exact(table)

# Example 1: Welch's t-test for unequal variances
In this first example, we are testing the biological hypothesis that a mutant genotype affects a phenotype we are measuring. Our statistical null hypothesis is that genotype has no effect on the measurement.

There are two variables in the experiment: Genotype and Measurement. Genotype is a categorical variable with two possible values: WT and Mutant. Measurement is a continuous numerical variable. Measurement is our response variable. Genotype is our treatment variable.

Based on these two variables, we will run a Student's two-sample t-test as long as we can meet the assumptions of that test: normally distributed responses within each treatment and equal variances between treatments. We have to look at our data to see if we have met these assumptions.

In [ ]:
# read fist data file into data frame object
data1 = pd.read_excel('dataset1.xlsx')
# note that the above line of code assumes the data file is in the same folder as the notebook


In [ ]:
# look at the first several rows of data
data1.head(5)

In [ ]:
# look at the first several rows of data
data1.tail(5)

In [ ]:
# look at the table size
data1.shape

In [ ]:
# look at summary information
data1.describe()

In the above code block we read the data in from an Excel file and saved it in a data frame variable called 'data1'. The data appears to have been read in correctly. It was organized in what is called a 'long format'. In the long format, each row is a replicate and the columns are the variables. In this case, each individual has a genotype and measurement. R does not know that we want Genotype to be a categorical variable so we have to tell it. Categorical variables are called 'factors' in R so we use the function as.factor() to make Genotype categorical. We confirmed that it worked using the str() function. The summary() function shows us some information about the data. From that we learn that the measurements range from 2 to 45.

We are interested in checking the assumptions of the t-test so we plot the data and calculate  summary statistics.

In [ ]:
# plot data as boxplot
sns.boxplot(data=data1,x='Genotype',y='Measurement')

In [ ]:
# pòot swarmplot
sns.swarmplot(data=data1,x='Genotype',y='Measurement')

In [ ]:
# plot violin-plot
sns.violinplot(data=data1,x='Genotype',y='Measurement')

In [ ]:
#binning
min_value = np.floor(data1['Measurement'].min())
max_value = np.ceil(data1['Measurement'].max())
print(min_value,max_value)

bins = np.linspace(min_value,max_value,10)
print(bins)

In [ ]:
# plot data as stacked histograms
fig,ax=plt.subplots(2,1,sharex=True)
ax[0].set_title('Mutant')
sns.histplot(data=data1[data1.Genotype=='Mutant'],x='Measurement',ax=ax[0],bins=bins)
ax[1].set_title('WT')
sns.histplot(data=data1[data1.Genotype=='WT'],x='Measurement',ax=ax[1],bins=bins)
plt.tight_layout()

In [ ]:
#summary statistics, stratified by genotype
data1.groupby('Genotype').agg(['count','mean','median','std','var'])

The plot() function by default makes a boxplot for this kind of data. From the boxplot you can tell that the variances are somewhat different between genotypes. The hist() function makes histograms. The stacked histograms give a sense for the shapes of each distribution. Although neither looks perfectly normal, neither is strongly skewed. The ddply() function (from the plyr package that we installed and loaded above) helps us organize our summary statistics for each genotype. The median and mean values for each genotype are very similar, confirming that the distributions are not highly skewed. From this information, we will say that the data have met the t-test assumption of normally distributed responses in each treatment. What about equal variances between treatments? The variances are an order of magnitude different, which violates the assumption of the t-test.

Instead of a Student's t-test, we can run a Welch's t-test which assumes normality but does not assume equal variances.

In [ ]:
data_wt = data1[data1.Genotype=='WT']['Measurement'].values
print('data_wt:',data_wt)

data_mutant = data1[data1.Genotype=='Mutant']['Measurement'].values
print('data_mutant:',data_mutant)

In [ ]:
# run Welch's t-test on data with alpha set to 0.05
test_result = st.ttest_ind(data_wt,data_mutant,equal_var=False,alternative='two-sided')
print(test_result)

In [ ]:
help(st.ttest_ind)

In [ ]:
res = stat()
res.ttest(df=data1,xfac='Genotype',res='Measurement',evar=False,test_type=2)
print(res.summary)

Our p-value is less than our alpha value of 0.05 so we reject the null hypothesis that genotype has no effect on our measured response. When reporting this result in a paper it is best to include t, df, and p-value. Here is what that might look like:

The mutant had significantly different measurements than wild type (Welch's t(2,0.05) = -3.54, df = 27.98, p-value < 0.00143).

The numbers in parentheses next to the t are 2 for a two-sided test (i.e. allowing the effect of the mutant to both increase or decrease the measurement) and 0.05 for the alpha value.

In addition to reporting the result in text, we can also show our data and our results using a figure. We are going to make a stripchart that includes the mean and 95% confidence intervals for each genotype. First we calculate the 95% confidence intervals around the means for each genotype.

We created three functions to help us calculate the 95% confidence intervals. The first function intmag95() calculates the magnitude of one half of the interval. It multiplies the SEM by the t statistic value that corresponds with an alpha value of 0.05. The upper95() and lower95() functions simply add or subtract the intmag95() value to/from the mean to get the actual value of the top and bottom of the confidence interval.

We then used the ddply() function again to run these functions we just wrote on the measurement values for each genotype. Looking at the table, you can see that the 95% confidence intervals for the two genotypes do not overlap. When 95% confidence intervals do not overlap, you can be sure that a statistical test will reject the null hypothesis of the categories have equal responses. And that is the case for us here.

Now we want to display the data, means, and confidence intervals.

The above stripchart conveys a lot of information to the reader. It shows the actual data points for each genotype and how they are distributed. It shows the mean values. And it shows the 95% confidence intervals which do not overlap and therefore convey that the mutant is significantly different than the wild type in measurement response.

# Example 2: Student's t-test on log transformed responses
In this second example we are working with a very similar dataset. The dataset has the same types of variables and we are interested in the same biological hypothesis. The difference comes when we inspect the data to see if it meets the assumptions of a Student's t-test.

In [ ]:
# read second data file into data frame object
data2 = pd.read_excel('dataset2.xlsx')
# look at the first several rows of data
data2.head(5)


In [ ]:
data2.shape

In [ ]:
# look at summary information
data2.describe()

In [ ]:
# plot data as boxplot
sns.boxplot(data=data2,x='Genotype',y='Measurement')

In [ ]:
#binning
min_value = data2['Measurement'].min()
max_value = data2['Measurement'].max()
print(min_value,max_value)

bins = np.linspace(min_value,max_value,15)
print(bins)

In [ ]:
# plot data as stacked histograms
fig,ax=plt.subplots(2,1,sharex=True)
ax[0].set_title('Mutant')
sns.histplot(data=data2[data2.Genotype=='Mutant'],x='Measurement',ax=ax[0],bins=bins)
ax[1].set_title('WT')
sns.histplot(data=data2[data2.Genotype=='WT'],x='Measurement',ax=ax[1],bins=bins)
plt.tight_layout()

In [ ]:
#summary statistics, stratified by genotype
data2.groupby('Genotype').agg(['count','mean','median','std','var'])

With regard to normality, the measurement responses appear to be somewhat skewed to the right. The median values are less than the mean values, consistent with a slight skew. This might be a violation of the assumption of normality. The responses for the mutant also appear to be slightly bimodal. Student's t-test is robust to small deviations from normality so it is unclear if these departures from normality will be a problem.

The variances are just about three-fold different. Student's t-test is robust to this level of difference in variance but only when the design is balanced. The sample sizes are somewhat different so again we are close to violating this assumption.

The skew in the response is the more serious violation because if it cannot be corrected then we will need to use a non-parametric test with lower power. Right-skewed distributions can sometimes be corrected using a natural log or reciprocal transformation. Note that the function log() in R defaults to the natural log, which is commonly written as ln.

In [ ]:
# ln transformation
data2['LnMeasurement'] = np.log(data2['Measurement'])


In [ ]:
#binning
min_value = data2['LnMeasurement'].min()
max_value = data2['LnMeasurement'].max()
print(min_value,max_value)

bins = np.linspace(min_value,max_value,15)
print(bins)

# plot data as stacked histograms
fig,ax=plt.subplots(2,1,sharex=True)
ax[0].set_title('Mutant')
sns.histplot(data=data2[data2.Genotype=='Mutant'],x='LnMeasurement',ax=ax[0],bins=bins)
ax[1].set_title('WT')
sns.histplot(data=data2[data2.Genotype=='WT'],x='LnMeasurement',ax=ax[1],bins=bins)
plt.tight_layout()

In [ ]:
#summary statistics, stratified by genotype
data2.groupby('Genotype')['LnMeasurement'].agg(['count','mean','median','std','var'])

Both transformations improved the shape of the distributions to be more normal in appearance. The natural log transformation also resulted in the variances being very similar between genotypes, while the reciprocal did not. The dataset with natural log transformations of the measurement responses meet the assumptions of Student's t-test.

In [ ]:
# run t-test on ln transformed data with alpha set to 0.05 and assumption of equal variance
data_wt = data2[data2.Genotype=='WT']['LnMeasurement']
data_mutant = data2[data2.Genotype=='Mutant']['LnMeasurement']


test_result = st.ttest_ind(data_wt,data_mutant,equal_var=False,alternative='two-sided')
print(test_result)

In [ ]:
# run t-test on ln transformed data with alpha set to 0.05 and assumption of equal variance
data_wt = data2[data2.Genotype=='WT']['Measurement']
data_mutant = data2[data2.Genotype=='Mutant']['Measurement']


test_result = st.ttest_ind(data_wt,data_mutant,equal_var=False,alternative='two-sided')
print(test_result)

The mutant had significantly different measurements than wild type (Student's t(2,0.05) = -12.299, df = 61, p-value < 2.2e-16).

The figure supports this conclusion with non-overlapping 95% confidence intervals.

# Example 3: Logistic regression for categorical response and two treatment variables
In the first two examples we had a continuous numerical response variable and a single categorical treatment variable. In this example we have a categorical response variable (Phenotype) and two categorical treatment variables (Genotype & Day).

Our biological hypothesis is that genotype (disomic vs trisomic) affects the proportion of cells that are ciliated. Data was collected over many days so we include Day as a so called 'nuisance' treatment variable. Our statistical null hypotheses are that the proportion of ciliated cells is the same across genotypes and that the proportion of ciliated cells is the same across days.

We can test these hypotheses using logistic regression in a generalize linear model and a series of Wald tests. This approach makes few assumptions about the structure of the data and the glm() function that we will use will warn us if our data are not meeting those assumptions.

In [ ]:
# read third data file into data frame object
data3 = pd.read_excel('dataset3.xlsx')
data3.head()
# from Domenico Galati et al 2018, Fig 1 (https://doi.org/10.1016/j.devcel.2018.07.008)


In [ ]:
data3_sel = pd.concat([data3[data3.Phenotype=='Ciliated'].head(300),data3[data3.Phenotype=='Not Ciliated'].head(520)])

In [ ]:
data3.shape

In [ ]:
data3.value_counts()

We will now run the glm() function on the data. We need to tell the glm() function which variable is the response variable and which variables are treatment variables. This is done in R by putting the response variable to the left of a tilda symbol and the treatment variables to the right of the tilda symbol with plus signs between treatment variables. The glm() function also needs to know what kind of regression we plan to do. Because our response variable is a binary categorical variable, the appropriate type of regression is called logistic regression. We specify this by telling glm() to use a binomial error distribution. The binomial distribution quantifies the probability of a binary outcome, like flipping a coin to get heads versus tails. Here, the binary outcome is ciliated versus not, and the logistic regression model is a way to allow the probability of being ciliated to depend on genotype and day.

In [ ]:
# https://realpython.com/logistic-regression-python/

# https://www.reneshbedre.com/blog/logistic-regression.html
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
data3['Genotype_numeric'] = data3['Genotype'].replace(['Disomic','Trisomic'],[0,1])
data3['Phenotype_numeric'] = data3['Phenotype'].replace(['Ciliated','Not Ciliated'],[0,1])


In [ ]:
# logistic regression model
import statsmodels.api as sm 
# get independent variables
X = data3[['Day','Genotype_numeric']]
# to get intercept -- this is optional
# X = sm.add_constant(X)
# get response variables
Y = data3[['Phenotype_numeric']]
# fit the model with maximum likelihood function
model = sm.Logit(endog=Y, exog=X).fit()

print(model.summary())

In [ ]:
np.exp(model.params['Genotype_numeric'])

In [ ]:
# build a generalized linear model of Phenotype with binomial errors and Genotype and Day as treatments
model = LogisticRegression()

x = data3[['Day','Genotype_numeric']]
y = data3['Phenotype_numeric']
model.fit(x,y)

In [ ]:
#coeff
model.coef_

In [ ]:
model.score(x,y)

In [ ]:
confusion_matrix(y, model.predict(x))

In [ ]:
#odds:
np.exp(model.coef_[0][1])

The section of the glm() function output that we are most interested in is the coefficients. The line that starts with "GenotypeTrisomic" tells us about the effect of genotype. The magnitude of the effect of being Trisomic vs Disomic is 1.2705. This coefficient is calculated as the natural log of the odds ratio. A Wald test was run on this coefficient to determine if it is a significant departure from what would be expected if there was no effect of genotype. The Wald test is based on z-scores. Dividing the coefficient by the standard error results in a z-score of 7.933 which has a p-value of 2.14e-15. So genotype has a very significant effect on the proportion of cells with cilia.

Simiarly, there were significant effects of what day the experiment was conducted on. Day2 and Day3 were both significantly different from Day1. Although the effect of day was not of interest, by including it as a variable in the model we were able to estimate and therefore control for significant variation in the proportion of cells with cilia across days. Controling for the day effect allowed us to more accurately estimate the effect and significance of genotype.

Next we want to display our results. It is very common to see this kind of data displayed as a bar graph with proportion or percentage on the y-axis and the different treatment categories on the x-axis. That is a fine way to display the data but it doesn't convey information about the hypothesis being tested.

An Odds Ratio plot is an excellent way to convey the effect of a binary categorical treatment variable on a binary categorical response variable. In this case the Odds Ratio = (Ciliated Disomic / Not Ciliated Disomic) / (Ciliated Trisomic / Not Ciliated Trisomic). As mentioned above, the coefficient estimated for GenotypeTrisomic is the natural log of the odds ratio. So the inverse natural log (i.e. 'e') of the coefficient is the odds ratio. Although it was not printed out to the screen, the glm() function also calculated the 95% confidence interval for the coefficient, which can also be converted to an odds ratio using the inverse natural log.

The figure conveys that the odds of having cilia in disomic cells is ~3.5 times higher than in trisomic cells. It also conveys that this difference significantly exceeds the null expectation of equal odds (the dashed line at 1), consistent with the small and significant p-value from the Wald test run by the glm() function.

# Example 4: Chi-Square contingency test on combined response and combined treatment groups
This example is similar to the previous example in that there is a categorical response variable and the treatment is categorical. However, in this example there is only one categorical treatment variable so instead of logistic regression we can run a chi-square contingency test.

The biological hypothesis is that loss of expression of the gene JMJD2A causes decreased expression of the gene Sox2 in developing chicken neural plate. The four treatments were control morpholino (Control-MO), translation blocking JMJD2A morpholino (JmjD2A-tbMO), splicing blocking JMJD2A morpholino (JmjD2A-sbMO), and splicing blocking JMJD2A morpholino plus JmjD2A full-length rescue vector. The response was Sox2 expression for each embryo categorized as full wild type expression (WT), mild decrease in expression (Mild), or strong decrease in expression (Strong). An appropriate null hypothesis is that Sox2 expression response is independent of JMJD2A treatment.

We can test our null hypothesis with a chi-square contingency test. The test calculates the expected counts of each combination of treatment and response assuming independence. The assumptions of the test are that none of the expected counts are less than 1 and <20% of the expected counts are less than 5. Let's look at the data to see if they meet these assumptions.

In [ ]:
# read fourth data file into data frame object
data4 = pd.read_excel('dataset4.xlsx')
data4.head()

In [ ]:
# create a contingency table
table = pd.crosstab(data4.Treatment,data4.Phenotype)
table

The table() function converted the long format data into a table of counts for combinations of treatment and phenotype (Sox2 expression). An efficient way to get the expected counts from the observed counts is to run the chisq.test() function and then print out the expected counts which it calculates.

In [ ]:
# run chi-square test with correct = F because this is not a 2x2 table
chi2test = st.chi2_contingency(table,correction=False)

In [ ]:
#print expected counts
chi2test[3]

Although the expected counts meet the first assumption that none can be less than 1, they do not meet the second assumption that <20% are less than 5. 5 out of 12 (41.7%) are less than 5.

There are two ways to handle this situation. One way is to increase sample sizes for each of the treatments. Another way is to consider if any of the treatment or response categories could be combined such that the assumption is satisfied while keeping the outcome of the hypothesis test interpretable and meaningful.

Given the biological hypothesis that loss of expression of the gene JMJD2A causes decreased expression of the gene Sox2, the two treatments intended to decrease expression of JMJD2A (JmjD2A-tbMO & JmjD2A-sbMO) could be combined and the two responses that involve decreased expression of Sox2 (Mild & Strong) could be combined.

In [ ]:
# this is an inelegant way of making a new counts table with JmjD2A-tbMO & JmjD2A-sbMO combined and Mild & Strong combined
#(data4.table.comb <- matrix(c(2+0,20,3+7+10+14,2+13,4+2,9), nrow = 3, byrow = T, dimnames = list(Treatment = c("Control-MO","JmjD2A-sbMO+tbMO","JmjD2A-sbMO+pCI-JmjD2A"), Phenotype = c("Mild+Strong","WT"))))

Now check the expected counts for this new observed counts table.

In [ ]:
# run chi-square test with correct = F because this is not a 2x2 table
#chitest4comb <- chisq.test(data4.table.comb, correct = F)
# print expected counts
#chitest4comb$expected

Now all expected values are 5 or more so the assumptions of the test have been met.

In [ ]:
#chitest4comb

The test resulted in a significant p-value. So we reject the null hypothesis that Sox2 expression is independent of treatments decreasing JMJD2A expression.

A table is an excellent way of displaying data from a contingency analysis. If a figure is desired, a mosaic plot is an appropriate type of graph.

In [ ]:
# set plot width & height (only needed for Jupyter)
#options(repr.plot.width=7, repr.plot.height=7)
# first make a table of proportions rather than counts
#(data4.table.comb.prop <- data4.table.comb / matrix(c(rep(sum(data4.table.comb[1,]),2), rep(sum(data4.table.comb[2,]),2), rep(sum(data4.table.comb[3,]),2)), nrow = 3, byrow = T))
# use barplot to make a mosaic plot
#barplot(t(data4.table.comb.prop), legend = T, width = c(sum(data4.table.comb[1,]), sum(data4.table.comb[2,]), sum(data4.table.comb[3,])), ylab="Proportion", args.legend = list(x = "top", bty="n"))

The mosaic plot conveys the differences in response proportions across treatments. Sample sizes for each treatment are reflected in the width of the bars. This figure could be improved by overlaying the counts in each bar.

# Example 5: ANOVA with Tukey-Kramer Post-hoc
In this example there is a categorical treatment variable and a continuous numerical response variable, just like the first two examples. However, in this example, the categorical treatment variable has four groups, three RNAi treatments and one control treatment.

The biological hypothesis is that RNAi against all three members of the ZLW gene family results in decreased fluorescence intensity of a reporter. A common mistake is to perform a series of t-tests, which does not control the false positive rate at or below 0.05. If all six pairwise comparisons were made using t-tests, the false positive rate would be capped at 6 x 0.05 = 0.3, which is quite high. A single ANOVA test can be run to determine if any of the mean responses across treatments are different. The null hypothesis is that the mean responses are the same across all treatments. If we reject this null hypothesis from the ANOVA, we can then run a Tukey-Kramer post-hoc analysis to determine which pairs of treatments are significantly different. And all of that can be done while keeping the false positive rate no higher than 0.05.

ANOVA has the same assumptions as t-tests: normality of responses within treatments and equal variances across treatments. Let's see if the data meet these assumptions.

In [ ]:
# read fifth data file into data frame object
data5 = pd.read_excel('dataset5.xlsx')

data5.head()

In [ ]:
data5.describe()

In [ ]:
data5.Treatment.value_counts()

In [ ]:
sns.boxplot(data=data5,x='Treatment',y='Intensity',color='lightblue')
sns.swarmplot(data=data5,x='Treatment',y='Intensity',color='b')

In [ ]:
#binning
min_value = data5['Intensity'].min()
max_value = data5['Intensity'].max()
print(min_value,max_value)

bins = np.linspace(min_value,max_value,15)
print(bins)

# plot data as stacked histograms
fig,ax=plt.subplots(4,1,sharex=True,figsize=(8,10))
i=0
for treatment in data5['Treatment'].unique():
    ax[i].set_title(treatment)
    sns.histplot(data=data5[data5.Treatment==treatment],x='Intensity',ax=ax[i],bins=bins)
    i = i+1

plt.tight_layout()

In [ ]:
#summary statistics, stratified by genotype
data5.groupby('Treatment')['Intensity'].agg(['count','mean','median','std','var'])

The distributions of responses for each treatment look approximately normally distributed. The variances are not equal but are all less than three fold different. ANOVA is robust to this level of difference in variance as long as the experiment is balanced. Sample sizes are large and almost identical so it is balanced. It appears we have met the assumptions of ANOVA.

In [ ]:
data_control = data5[data5.Treatment == 'control']['Intensity']
data_zlw1 = data5[data5.Treatment == 'ZLW1-RNAi']['Intensity']
data_zlw2 = data5[data5.Treatment == 'ZLW2-RNAi']['Intensity']
data_zlw3 = data5[data5.Treatment == 'ZLW3-RNAi']['Intensity']

#ANOVA
st.f_oneway(data_control,data_zlw1,data_zlw2,data_zlw3)

The aov() function, like the glm() function from the third example, takes an equation written as Response ~ Treatment. In our case that was Intensity ~ Treatment. The summary() function provides the results we seek. ANOVA runs what is called an F-test. Large values of F mean that the differences in responses amongst treatments are large compared to differences in responses within treatments. F values close to 1 mean that the differences in responses amongst treatments are similar to differences in responses within treatments. In our case, we got an F value of 28.68, which is very large and has a very significant p-value of 2.7e-16. We therefore reject the null hypothesis that the mean responses are the same across treatments.

If and only if you get a significant result from an ANOVA, you can run a Tukey-Kramer post-hoc analysis to determine which pairs of treatments differ significantly.

In [ ]:

# perform multiple pairwise comparison (Tukey's HSD)
# unequal sample size data, tukey_hsd uses Tukey-Kramer test
res = stat()
res.tukey_hsd(df=data5, res_var='Intensity', xfac_var='Treatment', anova_model='Intensity ~ C(Treatment)')
res.tukey_summary

The TukeyHSD() function performed the post-hoc analysis. The function returns a table with each of the six pairwise comparisons of treatments as the rows. The columns are 'diff', which is the difference in mean responses between the two treatments, 'lwr' and 'upr', which are the bounds of the 95% confidence interval for the 'diff', and 'p adj', which is the p-value from a q-test. In this case, five of the six possible pairwise comparisons had significantly different responses, demonstrated by a 'p adj' value less than 0.05 and a confidence interval that does not overlap zero. Only ZLW3 RNAi vs ZLW1 RNAi was not significant.

By using the ANOVA to Tukey-Kramer approach, we have kept the false positive rate at or below 0.05. We can draw the conclusion that RNAi of all three ZLW family genes significantly decreases fluorescence intensity of our reporter relative to a control. We can further say that RNAi of ZLW2 has a stronger effect on the fluorescence intensity of the reporter than ZLW1 and ZLW3.

For displaying our results in a figure, we can use a stripchart with mean and 95% confidence intervals overlayed like we did for examples 1 and 2. We have already calculated the mean responses for each treatment. Now we need to extract the 95% confidence intervals from the ANOVA result using the confint() function.

The function confint() gets the 95% confidence intervals for the first treatment and refers to this treatment as the intercept. In our case, that was ZLW1-RNAi. It then gets the confidence intervals for the other treatments relative to the "intercept" which is the mean response of ZLW1-RNAi. So in order to get the absolute confidence intervals rather than the relative confidence intervals, we have to add the mean response of ZLW1-RNAi (stored in stats5[1,4]) to all the remaining confidence interval lower and upper values. The second table printed has the absolute confidence intervals.

Now we can make the stripchart.

The figure shows the distribution of responses for each treatment which makes clear that these are overlapping distributions. The 95% confidence intervals around the mean response for each treatment however echo the results from the ANOVA and Tukey-Kramer post-hoc analysis, which is that the means are not equal and all of the treatments have significantly different responses with the exception of ZLW1-RNAi and ZLW3-RNAi. For ZLW1-RNAi and ZLW3-RNAi, the mean response of each treatment is contained within the confidence interval of the other treatment, making it clear that these are not significantly different treatments.